In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# #hide
# ! [ -e /content ] && pip install -Uqq fastbook
# import fastbook
# fastbook.setup_book()


In [7]:
#hide
#from fastbook import *
from IPython.display import display,HTML

In [8]:
import pandas as pd

In [9]:
%cd /content/drive/MyDrive/Colab Notebooks/SM-Assignment4-NLP/
path = '/content/drive/MyDrive/Colab Notebooks/SM-Assignment4-NLP/'
test_path = '/content/drive/MyDrive/Colab Notebooks/SM-Assignment4-NLP/test.csv'
train_path = '/content/drive/MyDrive/Colab Notebooks/SM-Assignment4-NLP/train.csv'

/content/drive/MyDrive/Colab Notebooks/SM-Assignment4-NLP


In [10]:
from fastai.text.all import *


In [11]:
import os

# Check if the path exists and list its contents
if os.path.exists(path):
    print("Files in path:", os.listdir(path))
else:
    print(f"Path does not exist: {path}")

Files in path: ['test.csv', 'train.csv', 'models']


In [12]:
# Load datasets
test_data = pd.read_csv(test_path, header=None)
train_data = pd.read_csv(train_path, header=None)

In [13]:
# Add column names
column_names = ['polarity', 'title', 'text']
test_data.columns = column_names
train_data.columns = column_names


In [14]:
# Define a custom get_items function that retrieves the text data
def get_items(df):
    return df['text'].values.tolist()  # Returns the text data as a list


In [15]:
# Check data
display(test_data.head())



,polarity,title,text
0,2,Great CD,"My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I'm in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life's hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing ""Who was that singing ?"""
1,2,One of the best game music soundtracks - for a game I didn't really play,"Despite the fact that I have only played a small portion of the game, the music I heard (plus the connection to Chrono Trigger which was great as well) led me to purchase the soundtrack, and it remains one of my favorite albums. There is an incredible mix of fun, epic, and emotional songs. Those sad and beautiful tracks I especially like, as there's not too many of those kinds of songs in my other video game soundtracks. I must admit that one of the songs (Life-A Distant Promise) has brought tears to my eyes on many occasions.My one complaint about this soundtrack is that they use guitar f..."
2,1,Batteries died within a year ...,"I bought this charger in Jul 2003 and it worked OK for a while. The design is nice and convenient. However, after about a year, the batteries would not hold a charge. Might as well just get alkaline disposables, or look elsewhere for a charger that comes with batteries that have better staying power."
3,2,"works fine, but Maha Energy is better","Check out Maha Energy's website. Their Powerex MH-C204F charger works in 100 minutes for rapid charge, with option for slower charge (better for batteries). And they have 2200 mAh batteries."
4,2,Great for the non-audiophile,"Reviewed quite a bit of the combo players and was hesitant due to unfavorable reviews and size of machines. I am weaning off my VHS collection, but don't want to replace them with DVD's. This unit is well built, easy to setup and resolution and special effects (no progressive scan for HDTV owners) suitable for many people looking for a versatile product.Cons- No universal remote."


In [16]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   polarity  400000 non-null  int64 
 1   title     399976 non-null  object
 2   text      400000 non-null  object
dtypes: int64(1), object(2)
memory usage: 9.2+ MB


In [17]:
example_review = train_data['text'][0]
example_review

'This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'

In [18]:
spacy = WordTokenizer()
toks = first(spacy([example_review]))
print(coll_repr(toks, 30))

(#81) ['This','sound','track','was','beautiful','!','It','paints','the','senery','in','your','mind','so','well','I','would','recomend','it','even','to','people','who','hate','vid','.','game','music','!','I'...]


In [19]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(example_review), 31))

(#88) ['xxbos','xxmaj','this','sound','track','was','beautiful','!','xxmaj','it','paints','the','senery','in','your','mind','so','well','i','would','recomend','it','even','to','people','who','hate','vid','.','game','music'...]


In [20]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [21]:
# ??replace_rep
# Here is a brief summary of what each does:

# fix_html:: Replaces special HTML characters with a readable version (IMDb reviews have quite a few of these)
# replace_rep:: Replaces any character repeated three times or more with a special token for repetition (xxrep), the number of times it's repeated, then the character
# replace_wrep:: Replaces any word repeated three times or more with a special token for word repetition (xxwrep), the number of times it's repeated, then the word
# spec_add_spaces:: Adds spaces around / and #
# rm_useless_spaces:: Removes all repetitions of the space character
# replace_all_caps:: Lowercases a word written in all caps and adds a special token for all caps (xxup) in front of it
# replace_maj:: Lowercases a capitalized word and adds a special token for capitalized (xxmaj) in front of it
# lowercase:: Lowercases all text and adds a special token at the beginning (xxbos) and/or the end (xxeos)

In [22]:
# Get first 2000 reviews
reviews = train_data['text'][:2000]
reviews2000 = train_data[:2000]

In [23]:
#def subword(sz):
#    sp = SubwordTokenizer(vocab_sz=sz)
#    sp.setup(txts)
#    return ' '.join(first(sp([txt]))[:40])

In [24]:
# Create token group to setup Numericalize
toks200 = reviews[:200].map(tkn)
toks200[0]

(#88) ['xxbos','xxmaj','this','sound','track','was','beautiful','!','xxmaj','it'...]

In [25]:
# Set up Numericalize which maps tokens to integers
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#816) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','i','and','to','a','of','it','is','this'...]"

In [26]:
# Apply Numricalize nums object and test it out
nums = num(toks)[:20]; nums

TensorText([  0, 343, 262,  25, 598,  31,   0,   0,   9,   0,  20,  93, 599,
             40, 127,   0,  51,   0,  17,  97])

In [27]:
# Apply Numericalize object to tokenized reviews
nums200 = toks200.map(num)

In [28]:
# Pass numericalized reviews to LM Data Loader
dl = LMDataLoader(nums200)

In [29]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [30]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj this sound track was beautiful ! xxmaj it xxunk the xxunk in your mind so well i would'

In [31]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj this sound track was beautiful ! xxmaj it xxunk the xxunk in your mind so well i would xxunk'

In [32]:
###################################################################
# Example Language model that predicts the next word of a document
# Modify DataBlock to work with csv data

dls_lm = DataBlock(
    blocks=TextBlock.from_df('text', is_lm=True),
    get_x=ColReader('text'),
    splitter=RandomSplitter(0.1)
).dataloaders(reviews2000, path=path, bs=128, seq_len=80)

In [33]:
# Show batch
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos i ordered according to the size chart for my measurements and it was way to small ! ! xxmaj very xxunk returned it , xxbos i am very xxunk about the xxunk in my xxmaj xxunk -- and for its small size this beautiful little xxmaj bible is very easy to read . i have the xxmaj xxunk xxmaj leather xxmaj bound edition which is beautifully bound and finished . xxmaj this is the xxunk "" xxunk "" version","i ordered according to the size chart for my measurements and it was way to small ! ! xxmaj very xxunk returned it , xxbos i am very xxunk about the xxunk in my xxmaj xxunk -- and for its small size this beautiful little xxmaj bible is very easy to read . i have the xxmaj xxunk xxmaj leather xxmaj bound edition which is beautifully bound and finished . xxmaj this is the xxunk "" xxunk "" version --"
1,"worry about the battery aspect . xxmaj then xxunk my ceiling one broke somehow . xxmaj so i bought this one , plugged it in , in the xxunk near my xxunk … … he said to do that so you can hear it quickly , and xxunk . xxmaj it 's doing it 's thing . a little xxunk , but actually one of the cheaper ones . xxmaj he xxunk ! xxbos xxmaj we have been giving this","about the battery aspect . xxmaj then xxunk my ceiling one broke somehow . xxmaj so i bought this one , plugged it in , in the xxunk near my xxunk … … he said to do that so you can hear it quickly , and xxunk . xxmaj it 's doing it 's thing . a little xxunk , but actually one of the cheaper ones . xxmaj he xxunk ! xxbos xxmaj we have been giving this to"


In [34]:
# Load pretrained Language model
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [35]:
# Train new layers with Amazon review data while keeping pretrained layers frozen
# Use a learning rate of 2e-2 = 0.02
%%time
learn.fit_one_cycle(1, 2e-2)

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.198154,3.981412,0.260024,53.592674,08:00


CPU times: user 7min 18s, sys: 30.3 s, total: 7min 49s
Wall time: 8min


In [36]:
# Save state of model after 1 epoch
learn.save('1epoch')

Path('/content/drive/MyDrive/Colab Notebooks/SM-Assignment4-NLP/models/1epoch.pth')

In [37]:
# Load model back
learn = learn.load('1epoch')

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device)


In [38]:
# Unfreeze the entire model and fine-tune the whole model
%%time
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.808066,3.868252,0.269455,47.858658,08:00
1,3.714000,3.794065,0.277330,44.436691,08:02
2,3.592183,3.788562,0.275935,44.192787,07:51
3,3.431548,3.834260,0.273472,46.259205,07:55
4,3.230877,3.904286,0.272217,49.614651,07:55
5,3.007598,4.030922,0.264104,56.312805,07:47
6,2.774320,4.177046,0.256487,65.173035,07:53
7,2.554904,4.285951,0.253202,72.671631,07:48
8,2.374567,4.326701,0.250642,75.694168,07:48
9,2.229893,4.340777,0.250049,76.767159,07:48


CPU times: user 1h 13min 7s, sys: 4min 56s, total: 1h 18min 4s
Wall time: 1h 18min 51s


In [39]:
# Save the learning model not including the final layer
learn.save_encoder('finetuned')

In [40]:
#TEXT = "I liked this movie because"
#N_WORDS = 40
#N_SENTENCES = 2
#preds = [learn.predict(TEXT, N_WORDS, temperature=0.75)
#         for _ in range(N_SENTENCES)]

In [41]:
#print("\n".join(preds))

In [42]:
# Creating the Classifier DataLoader
dls_clas = DataBlock(
    blocks=(TextBlock.from_df('text', vocab=dls_lm.vocab),CategoryBlock),
    get_x = ColReader('text'),
    get_y = ColReader('polarity'),
    splitter=RandomSplitter(0.1)
).dataloaders(reviews2000, path=path, bs=128, seq_len=72)

In [43]:
# Show batch to check data
dls_clas.show_batch(max_n=3)

,text,category
0,xxbos xxup first xxup of xxup all xxup how xxup come xxup all xxup ya'll xxup be xxup xxunk ' xxup no xxup limit ( for all ya'll that did not xxunk xxunk ' xxunk ai n't for you ) xxup what xxup ya'll xxup xxunk xxup about xxup cash xxup money xxup can xxup never xxup get xxup with xxup no xxup limit xxup and xxup that xxup is a xxup fact … .. xxup and xxup you xxup all xxup be xxup xxunk xxup about xxup they xxup be xxup xxunk xxup the xxup say xxup shit xxup what xxup about xxup xxunk xxup sorry xxup lil ' xxup wayne xxup be xxup sayin xxup the xxup same xxup shit xxup from xxup juvenile 's xxup album xxup and xxup b.g . 's xxup xxunk xxup xxunk xxup ha ) xxup what xxup kind xxup of xxup song,1
1,"xxbos xxmaj this is the second best xxmaj cash xxmaj money album xxmaj i 've ever heard next to xxmaj juvenile 's 400 xxmaj degreez which is xxunk a 5 xxmaj star cd too . xxmaj the xxmaj hot xxmaj boys put out an album with like 5 or 6 really strong songs . xxmaj and as bunch of other average or a little better songs . xxmaj not really any horrible songs at all . xxmaj the xxmaj best xxmaj songs are the main 4 "" i xxmaj need a xxmaj hot xxmaj girl "" , "" we xxmaj on xxmaj fire "" , "" respect xxmaj my xxmaj mind "" & "" xxunk & xxmaj xxunk "" . xxmaj those 4 songs were the songs i listened to every morning for like months and months . xxmaj and i have over 200 cd 's ! xxmaj so this",2
2,"xxbos i was extremely lucky to have found this , because it is a ) an import & b ) limited edition … especially lucky to find just thrown in the kylie section of a xxmaj xxunk xxmaj borders ! xxmaj ok , here it is track - by - xxunk . xxmaj chocolate [ radio xxunk this cuts the album version of the song by one minute and two seconds , it 's better . xxmaj her vocals are improved , & more of an arrangement is used . 10 / xxunk . xxmaj city xxmaj games : this extreme xxunk is fabulous . xxmaj very xxup r&b ; pop . 10 / xxunk . xxmaj chocolate [ xxunk xxmaj xxunk xxmaj xxunk xxmaj remix ] : a club dub . xxmaj cool when you have so much energy in you that you have to dance it out ,",2


In [44]:
# Create mini-batch with the first 10 reviews
nums_samp = toks200[:10].map(num)

In [45]:
# Show hiow many tokens each of these reviews has
nums_samp.map(len)

,text
0,88
1,110
2,178
3,188
4,119
5,177
6,181
7,121
8,116
9,68


In [46]:
# Create a model to classify reviews
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [47]:
# Load the encoder from the fine-tuned language model
learn = learn.load_encoder('finetuned')

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(join_path_file(file,self.path/self.model_

In [51]:
# Run Classifier model for 5 epochs with a learning rate of 2e-3 = 0.003
# Resulted in an accuracy of 84%
%%time
learn.fit_one_cycle(5, 2e-3)

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


epoch,train_loss,valid_loss,accuracy,time
0,0.323891,0.460034,0.835000,03:17
1,0.320772,0.452297,0.840000,03:15
2,0.320362,0.454909,0.840000,03:16
3,0.319180,0.453958,0.840000,03:14
4,0.320606,0.451474,0.840000,03:35


CPU times: user 15min 9s, sys: 53.9 s, total: 16min 2s
Wall time: 16min 38s
